1. `boolean tryAcquire(int arg)`
2. `boolean tryRelease(int arg)`


1. `int tryAcquireShared(int arg)`
2. `boolean tryReleaseShared(int arg)`

### 二. 如何使用AbstractQueuedSynchronizer构建同步类
<img src="img/aqsdemo.png" width="70%">


### 三. 几个问题: 
#### 1. FutureTask如何实现阻塞版的get(), 等待任务完毕才能返回
1. java1.8中实现的FutureTask使用LockSupport实现任务的等待  
    * (1) LockSupport中定义了线程阻塞,唤醒的方法; 定义了获取线程阻塞在哪个object上等方法
    * (2) `Unsafe`类的作用:
        1. 阻塞或唤醒线程 : 
            * `public native  void park(boolean isAbsolute,long time)`
            * `public native  void unpark(Object thread)`
        2. 获得内存地址上的值 : 
            * `public native double getDoubleVolatile(Object o,long offset)`
            * `public native float getFloat(long address)`
            * `public native float getFloat(Object o,long offset)`
        3. 修改内存地址的值
            * `public native  void putDouble(long address,double x)`
            * `public native  void putDouble(Object o,long offset,double x)`

     上面几个方法中的offset值内存的偏移地址, 此外Unsafe中定义了不同数据类型的单个偏移地址. 比如
     ```java
     ARRAY_LONG_INDEX_SCALE = theUnsafe.arrayIndexScale(long[].class);
     ```
     
2. FutureTask阻塞版get()的实现
    1. FutureTask对任务的状态做了定义, 每个状态都是一个volatile的int值
       <img src="img/futuretask1.png" width="40%">
    2. 再来看一下FutureTask内部的一些属性:
        ```java
        /** 定义任务 */
        private Callable<V> callable;
        /** 任务执行的结果, 供get()发挥 */
        private Object outcome; // non-volatile, protected by state reads/writes
        /** 执行任务的线程 */
        private volatile Thread runner;
        /** 等待获取get的线程栈 */
        private volatile WaitNode waiters;
        ```
    2. FutureTask的run方法 :   
       在线程池中的本地线程执行Callable的call方法
       ```java
        public void run() {
            if (state != NEW ||
                // 1. CAS设置该FutureTask的runner对象为当前线程 runnerOffset=UNSAFE.objectFieldOffset(k.getDeclaredField("runner")
                !UNSAFE.compareAndSwapObject(this, runnerOffset, null, Thread.currentThread()))
                return;
            try {
                Callable<V> c = callable;
                if (c != null && state == NEW) {
                    V result;
                    boolean ran;
                    try {
                        result = c.call();  // 2. 执行任务
                        ran = true;
                    } catch (Throwable ex) {
                        result = null;
                        ran = false;
                        setException(ex);
                    }
                    if (ran)
                        set(result);
                }
            } finally {
                // runner must be non-null until state is settled to prevent concurrent calls to run()
                runner = null;
                // state must be re-read after nulling runner to prevent leaked interrupts
                int s = state;
                if (s >= INTERRUPTING)
                    handlePossibleCancellationInterrupt(s);
            }
        }
       ```
    2. get()方法首先判断当前任务的任务的状态:  
        * 如果还未运行完(<=Compiled), 则自旋等待任务运行完(awaitDone): 否则返回运行结果
            1. 在等待栈中加入一个新的等待节点WorkNode
            ```java
            static final class WaitNode {
                volatile Thread thread;
                volatile WaitNode next;
                WaitNode() { thread = Thread.currentThread(); }
            }
            ```
            2. 如果任务状态是COMPLETING, 表示还未运行完, 进入自选等待(while(true){Thread.yield})
            ```java
            private int awaitDone(boolean timed, long nanos) throws InterruptedException {
                final long deadline = timed ? System.nanoTime() + nanos : 0L;  // 如果是有闲时间等待的话计算deadline
                WaitNode q = null;
                boolean queued = false;
                for (;;) {
                    if (Thread.interrupted()) {
                        removeWaiter(q);
                        throw new InterruptedException();
                    }

                    int s = state;
                    if (s > COMPLETING) {
                        if (q != null)
                            q.thread = null;
                        return s;
                    }
                    else if (s == COMPLETING) // 还未运行完, 自旋等待
                        Thread.yield();
                    else if (q == null)
                        q = new WaitNode();
                    else if (!queued)
                        //像等待栈中加入一个节点. waitersOffset = UNSAFE.objectFieldOffset(k.getDeclaredField("waiters"));
                        queued = UNSAFE.compareAndSwapObject(this, waitersOffset, q.next = waiters, q);
                    else if (timed) {
                        nanos = deadline - System.nanoTime();
                        if (nanos <= 0L) {
                            removeWaiter(q);
                            return state;
                        }
                        LockSupport.parkNanos(this, nanos);
                    }
                    else
                        LockSupport.park(this);
                    }
                }
            ```
    3.  cancel取消任务:
        因为runner属性记录了当前运行任务的线程, 所以调用运行任务线程的interupt()方法.所以, 只有在Callable的call()方法中设计了响应中断, 才能让cancel()方法生效. 
        ```java
        while(Thread.currentThread().isInterrupted() && ! flag)
        ```